In [334]:
from sympy import *

In [335]:
divVs, divqD, Pt0, Pf0, phi_0 = symbols('divVs, divqD, Pt0, Pf0, phi_0')
K_s, K_f, K_phi, eta_phi, sin_psi, gamma = symbols('K_s, K_f, K_phi, eta_phi, sin_psi, gamma')
Pt, Pf, dt = symbols('Pt, Pf, dt')
phi = symbols('phi')

In [336]:
elastic = 1
viscous = 1
plastic = 1

In [337]:
dPtdt  = (Pt-Pt0)/dt
dPfdt  = (Pf-Pf0)/dt
dphidt = elastic/K_phi*(dPfdt-dPtdt) + viscous/eta_phi*(Pf - Pt)
# phi    = phi_0 + dphidt*dt

# fpt = divVs + (dPtdt-phi*dPfdt)/K_s/(1-phi) + ( (dPtdt-dPfdt)/K_phi + (Pt-Pf)/eta_phi - gamma*sin_psi)/(1-phi)
dlnrhosdt = elastic/K_s/(1-phi) * (dPtdt - phi*dPfdt) 
fpt       = dlnrhosdt - dphidt/(1-phi) + divVs
fpt       = fpt.simplify()

# fpf = divqD + divVs*phi + gamma*sin_psi + (Pf-Pt)/eta_phi + (dPfdt - dPtdt)/K_phi + phi*dPfdt/K_f
dlnrhofdt = dPfdt/K_f
fpf       = elastic*phi*dlnrhofdt + dphidt + phi*divVs + divqD
fpf       = fpf.simplify()

fphi = phi    - (phi_0 + dphidt*dt)

display(fpt)
display(fpf)

(K_phi*K_s*divVs*dt*eta_phi*(phi - 1) + K_phi*eta_phi*(-Pt + Pt0 + phi*(Pf - Pf0)) + K_s*(K_phi*dt*(Pf - Pt) + eta_phi*(Pf - Pf0 - Pt + Pt0)))/(K_phi*K_s*dt*eta_phi*(phi - 1))

(K_f*K_phi*dt*eta_phi*(divVs*phi + divqD) + K_f*K_phi*dt*(Pf - Pt) + K_f*eta_phi*(Pf - Pf0 - Pt + Pt0) + K_phi*eta_phi*phi*(Pf - Pf0))/(K_f*K_phi*dt*eta_phi)

In [338]:
P_trial = solve([fpt, fpf], [Pt, Pf])

In [339]:
julia_code(P_trial[Pt].simplify())

'(-K_f .* K_phi .* K_s .* divVs .* dt .^ 2 - K_f .* K_phi .* K_s .* divqD .* dt .^ 2 - K_f .* K_phi .* Pf0 .* dt .* phi + K_f .* K_phi .* Pt0 .* dt - K_f .* K_phi .* divVs .* dt .* eta_phi .* phi .^ 2 - K_f .* K_phi .* divqD .* dt .* eta_phi .* phi - K_f .* K_s .* divVs .* dt .* eta_phi - K_f .* K_s .* divqD .* dt .* eta_phi - K_f .* Pt0 .* eta_phi .* phi + K_f .* Pt0 .* eta_phi + K_phi .* K_s .* Pf0 .* dt .* phi + K_phi .* K_s .* divVs .* dt .* eta_phi .* phi .^ 2 - K_phi .* K_s .* divVs .* dt .* eta_phi .* phi + K_phi .* Pt0 .* eta_phi .* phi + K_s .* Pt0 .* eta_phi .* phi) ./ (-K_f .* K_phi .* dt .* phi + K_f .* K_phi .* dt - K_f .* eta_phi .* phi + K_f .* eta_phi + K_phi .* K_s .* dt .* phi + K_phi .* eta_phi .* phi + K_s .* eta_phi .* phi)'

In [340]:
julia_code(P_trial[Pf].simplify())

'(-K_f .* K_phi .* K_s .* divVs .* dt .^ 2 - K_f .* K_phi .* K_s .* divqD .* dt .^ 2 - K_f .* K_phi .* Pf0 .* dt .* phi + K_f .* K_phi .* Pt0 .* dt - K_f .* K_phi .* divVs .* dt .* eta_phi .* phi - K_f .* K_phi .* divqD .* dt .* eta_phi - K_f .* K_s .* divVs .* dt .* eta_phi - K_f .* K_s .* divqD .* dt .* eta_phi - K_f .* Pf0 .* eta_phi .* phi + K_f .* Pf0 .* eta_phi + K_phi .* K_s .* Pf0 .* dt .* phi + K_phi .* Pf0 .* eta_phi .* phi + K_s .* Pf0 .* eta_phi .* phi) ./ (-K_f .* K_phi .* dt .* phi + K_f .* K_phi .* dt - K_f .* eta_phi .* phi + K_f .* eta_phi + K_phi .* K_s .* dt .* phi + K_phi .* eta_phi .* phi + K_s .* eta_phi .* phi)'

In [341]:
julia_code(solve(fphi, phi)[0] .simplify())

'(K_phi .* dt .* (Pf - Pt) + K_phi .* eta_phi .* phi_0 + eta_phi .* (Pf - Pf0 - Pt + Pt0)) ./ (K_phi .* eta_phi)'

In [342]:
dPtdt  = (Pt-Pt0)/dt
dPfdt  = (Pf-Pf0)/dt
dphidt = elastic/K_phi*(dPfdt-dPtdt) + viscous/eta_phi*(Pf - Pt) + plastic*gamma*sin_psi
# phi    = phi_0 + dphidt*dt

dlnrhosdt = elastic/K_s/(1-phi) * (dPtdt - phi*dPfdt) 
fpt = dlnrhosdt - dphidt/(1-phi) + divVs
fpt = fpt.simplify()

dlnrhofdt = dPfdt/K_f
fpf  = elastic*phi*dlnrhofdt + dphidt + phi*divVs + divqD
fpf = fpf.simplify()

P_corr = solve([fpt, fpf], [Pt, Pf])
P_corr

{Pt: (K_f*K_phi*K_s*divVs*dt**2 + K_f*K_phi*K_s*divqD*dt**2 + K_f*K_phi*Pf0*dt*phi - K_f*K_phi*Pt0*dt + K_f*K_phi*divVs*dt*eta_phi*phi**2 + K_f*K_phi*divqD*dt*eta_phi*phi + K_f*K_phi*dt*eta_phi*gamma*phi*sin_psi + K_f*K_s*divVs*dt*eta_phi + K_f*K_s*divqD*dt*eta_phi + K_f*Pt0*eta_phi*phi - K_f*Pt0*eta_phi - K_phi*K_s*Pf0*dt*phi - K_phi*K_s*divVs*dt*eta_phi*phi**2 + K_phi*K_s*divVs*dt*eta_phi*phi - K_phi*K_s*dt*eta_phi*gamma*phi*sin_psi - K_phi*Pt0*eta_phi*phi - K_s*Pt0*eta_phi*phi)/(K_f*K_phi*dt*phi - K_f*K_phi*dt + K_f*eta_phi*phi - K_f*eta_phi - K_phi*K_s*dt*phi - K_phi*eta_phi*phi - K_s*eta_phi*phi),
 Pf: (K_f*K_phi*K_s*divVs*dt**2 + K_f*K_phi*K_s*divqD*dt**2 + K_f*K_phi*Pf0*dt*phi - K_f*K_phi*Pt0*dt + K_f*K_phi*divVs*dt*eta_phi*phi + K_f*K_phi*divqD*dt*eta_phi + K_f*K_phi*dt*eta_phi*gamma*sin_psi + K_f*K_s*divVs*dt*eta_phi + K_f*K_s*divqD*dt*eta_phi + K_f*Pf0*eta_phi*phi - K_f*Pf0*eta_phi - K_phi*K_s*Pf0*dt*phi - K_phi*Pf0*eta_phi*phi - K_s*Pf0*eta_phi*phi)/(K_f*K_phi*dt*phi - K_f*K

In [343]:
P_trial[Pt]

(K_f*K_phi*K_s*divVs*dt**2 + K_f*K_phi*K_s*divqD*dt**2 + K_f*K_phi*Pf0*dt*phi - K_f*K_phi*Pt0*dt + K_f*K_phi*divVs*dt*eta_phi*phi**2 + K_f*K_phi*divqD*dt*eta_phi*phi + K_f*K_s*divVs*dt*eta_phi + K_f*K_s*divqD*dt*eta_phi + K_f*Pt0*eta_phi*phi - K_f*Pt0*eta_phi - K_phi*K_s*Pf0*dt*phi - K_phi*K_s*divVs*dt*eta_phi*phi**2 + K_phi*K_s*divVs*dt*eta_phi*phi - K_phi*Pt0*eta_phi*phi - K_s*Pt0*eta_phi*phi)/(K_f*K_phi*dt*phi - K_f*K_phi*dt + K_f*eta_phi*phi - K_f*eta_phi - K_phi*K_s*dt*phi - K_phi*eta_phi*phi - K_s*eta_phi*phi)

In [344]:
dPt = ((P_corr[Pt] - P_trial[Pt])).simplify()
julia_code(dPt)

'K_phi .* dt .* eta_phi .* gamma .* phi .* sin_psi .* (-K_f + K_s) ./ (-K_f .* K_phi .* dt .* phi + K_f .* K_phi .* dt - K_f .* eta_phi .* phi + K_f .* eta_phi + K_phi .* K_s .* dt .* phi + K_phi .* eta_phi .* phi + K_s .* eta_phi .* phi)'

In [345]:
dPf = ((P_corr[Pf] - P_trial[Pf])).simplify()
julia_code(dPf)

'K_f .* K_phi .* dt .* eta_phi .* gamma .* sin_psi ./ (K_f .* K_phi .* dt .* phi - K_f .* K_phi .* dt + K_f .* eta_phi .* phi - K_f .* eta_phi - K_phi .* K_s .* dt .* phi - K_phi .* eta_phi .* phi - K_s .* eta_phi .* phi)'